In [ ]:
from google.colab import drive
drive.mount('/content/drive')#

Mounted at /content/drive


In [ ]:
!pip install imbalanced-learn
!pip install contractions
!pip install torch torchvision
!pip install bayesian-optimization

     |████████████████████████████████| 286kB 10.7MB/s 
     |████████████████████████████████| 327kB 36.1MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85399 sha256=a57feafdc977f23fe46a80fd11bedd634b362e2f89f6e6b3bc49f5ed82093dd5
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=b85c5b87fd3cbe2a8d85c5e0627cb18bb6e61a7aff8defca21e8027a7b05ecc2
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
import json
import numpy as np
import pandas as pd
import contractions
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import re
import matplotlib.pyplot as plt

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

import string
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import gensim

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

torch.manual_seed(0)
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
    
print("Using GPU: {}".format(use_cuda))

from bayes_opt import BayesianOptimization

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Using GPU: True


In [ ]:
with open('/content/drive/My Drive/Individual Project/cleaned_depression_data.json') as f:
    depression_data = json.load(f)
  
with open('/content/drive/My Drive/Individual Project/cleaned_anxiety_data.json') as f:
    anxiety_data = json.load(f)

with open('/content/drive/My Drive/Individual Project/cleaned_bipolar_data.json') as f:
    bipolar_data = json.load(f)

with open('/content/drive/My Drive/Individual Project/cleaned_bpd_data.json') as f:
    bpd_data = json.load(f)

with open('/content/drive/My Drive/Individual Project/cleaned_ocd_data.json') as f:
    ocd_data = json.load(f)

with open('/content/drive/My Drive/Individual Project/cleaned_control_data.json') as f:
    control_data = json.load(f)

In [ ]:
len(depression_data)

121668

In [ ]:
len(anxiety_data)

119288

In [ ]:
len(bipolar_data)

81133

In [ ]:
len(bpd_data)

119714

In [ ]:
len(ocd_data)

55656

In [ ]:
len(control_data)

228974

In [ ]:
lengths = []
for i in range(0, len(control_data)):
  lengths.append(len(control_data[i]['selftext']))

print(min(lengths))
print(np.percentile(lengths, 25))
print(np.percentile(lengths, 50))
print(np.percentile(lengths, 75))
print(max(lengths))

51
79.0
122.0
204.0
999


In [ ]:
print(ocd_data[678]['selftext'])

Every so often, I'll get the idea that I'm being watched on camera in my home. It feels very similar to the Truman Show delusion. However, in my case, instead of feeling like I am on a reality show or that a large audience can see me, I feel that only one specific person is watching me. The specific person has changed over the years, but it's always only one person at a time and is someone that I know well in real life. 


In [ ]:
selftext = []
subreddit = []

for i in range(0, len(depression_data)):
  selftext.append(depression_data[i]['selftext'])
  subreddit.append('depression')

for i in range(0, len(anxiety_data)):
  selftext.append(anxiety_data[i]['selftext'])
  subreddit.append('anxiety')

for i in range(0, len(ocd_data)):
  selftext.append(ocd_data[i]['selftext'])
  subreddit.append('ocd')

for i in range(0, len(bipolar_data)):
  selftext.append(bipolar_data[i]['selftext'])
  subreddit.append('bipolar')

for i in range(0, len(bpd_data)):
  selftext.append(bpd_data[i]['selftext'])
  subreddit.append('bpd')

for i in range(0, len(control_data)):
  selftext.append(control_data[i]['selftext'])
  subreddit.append('control')

print(len(selftext))
print(len(subreddit))

726433
726433


In [ ]:
df = pd.DataFrame({'selftext':selftext, 'subreddit':subreddit})

In [ ]:
df = df.sample(frac = 1)

In [ ]:
len(df)

726433

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['selftext'], df['subreddit'], test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [ ]:
train_df = pd.DataFrame({'selftext':x_train, 'subreddit':y_train})
val_df = pd.DataFrame({'selftext':x_val, 'subreddit':y_val})
test_df = pd.DataFrame({'selftext':x_test, 'subreddit':y_test})

In [ ]:
train_df.to_csv('/content/drive/My Drive/Individual Project/train.csv', index=False)
val_df.to_csv('/content/drive/My Drive/Individual Project/validation.csv', index=False)
test_df.to_csv('/content/drive/My Drive/Individual Project/test.csv', index=False)

In [ ]:
new_df = pd.read_csv('/content/drive/My Drive/Individual Project/test.csv', lineterminator='\n')

In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/Individual Project/train.csv', lineterminator='\n')
val_df = pd.read_csv('/content/drive/My Drive/Individual Project/validation.csv', lineterminator='\n')
test_df = pd.read_csv('/content/drive/My Drive/Individual Project/test.csv', lineterminator='\n')

In [ ]:
print(len(control_data))
print(len(anxiety_data))
print(len(depression_data))
print(len(ocd_data))
print(len(bpd_data))
print(len(bipolar_data))

NameError: ignored

In [ ]:
def remove_contractions(text):
  expanded_words = []    
  for word in text.split():
    # using contractions.fix to expand the shotened words
    expanded_words.append(contractions.fix(word))   
    
  expanded_text = ' '.join(expanded_words)

  return expanded_text

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
def preprocess_text(text):
  text = remove_contractions(text)
  tokens = nltk.word_tokenize(text)
  tokens = [token.lower() for token in tokens]
  tokens = list(filter(lambda x: x not in string.punctuation, tokens))
  tokens = list(filter(lambda x: not x.isnumeric(), tokens))
  stop_words = set(stopwords.words('english'))
  tokens = list(filter(lambda x: x not in stop_words, tokens))
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]

  return ' '.join(tokens)

In [ ]:
# remove all characters which are not letters
def sanitise_text(text):
  words = text.split(' ')
  cleaned_words = []
  for word in words:
    word = re.sub("[^a-zA-Z]+", "", word)
    word = word.strip()
    cleaned_words.append(word)

  return " ".join(cleaned_words).strip()

In [ ]:
train_df['preprocessed_selftext'] = train_df['selftext'].map(lambda x: preprocess_text(x))
val_df['preprocessed_selftext'] = val_df['selftext'].map(lambda x: preprocess_text(x))
test_df['preprocessed_selftext'] = test_df['selftext'].map(lambda x: preprocess_text(x))

In [ ]:
train_df.to_csv('/content/drive/My Drive/Individual Project/train_preprocessed.csv', index=False)
val_df.to_csv('/content/drive/My Drive/Individual Project/validation_preprocessed.csv', index=False)
test_df.to_csv('/content/drive/My Drive/Individual Project/test_preprocessed.csv', index=False)

In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/Individual Project/train_preprocessed.csv', lineterminator='\n')
val_df = pd.read_csv('/content/drive/My Drive/Individual Project/validation_preprocessed.csv', lineterminator='\n')
test_df = pd.read_csv('/content/drive/My Drive/Individual Project/test_preprocessed.csv', lineterminator='\n')

In [ ]:
train_df.head(20)

,selftext,subreddit,preprocessed_selftext
0,And just when everything was finally starting ...,depression,everything finally start work struggle past ye...
1,"We care for each other, but they these types o...",bipolar,care type mental illness something childish ha...
2,Hello everyone! This seemed like the best pla...,control,hello everyone seem like best place ask advice...
3,My best friend is 23. She has repeatedly told ...,control,best friend repeatedly told go mother wish mot...
4,I have trust issues due to past relationships ...,depression,trust issue due past relationship feel like fi...
5,Hello All I am currently facing a hearing with...,control,hello currently face hearing allege student co...
6,My father died in mid December 33 years ago an...,depression,father die mid december year ago year easy dea...
7,In the last 6 years something has broken in me...,depression,last year something broken feel people soon le...
8,"Age: 28 Sex: Female Height: 5'6"" Weight: Ob...",control,age sex female height weight obese sure rele...
9,Hi Im looking to gain a little weight anyone k...,control,hi look gain little weight anyone know good ap...


In [ ]:
train_df = train_df.dropna()
val_df = val_df.dropna()
test_df = test_df.dropna()

In [ ]:
print(len(train_df))
print(len(val_df))
print(len(test_df))

223307
24812
27569


In [ ]:
print(len(train_df[train_df['subreddit'] == 'control']))
print(len(train_df[train_df['subreddit'] == 'anxiety']))
print(len(train_df[train_df['subreddit'] == 'depression']))
print(len(train_df[train_df['subreddit'] == 'bpd']))
print(len(train_df[train_df['subreddit'] == 'bipolar']))

74409
36732
38752
34150
39264


In [ ]:
train_df['preprocessed_selftext'] = train_df['preprocessed_selftext'].map(lambda x: sanitise_text(x))
val_df['preprocessed_selftext'] = val_df['preprocessed_selftext'].map(lambda x: sanitise_text(x))
test_df['preprocessed_selftext'] = test_df['preprocessed_selftext'].map(lambda x: sanitise_text(x))

In [ ]:
test_df['preprocessed_selftext'].iloc[11]

'impulse kick get anxious like must'

In [ ]:
undersample = RandomUnderSampler(sampling_strategy = {'control': 34000, 'depression': 34000, 'anxiety': 34000, 'bpd': 34000, 'bipolar': 34000}, random_state=42)
X_under, y_under = undersample.fit_resample(train_df[['preprocessed_selftext']], train_df['subreddit'])
train_df_balanced = pd.DataFrame({'preprocessed_selftext':X_under[ : , 0], 'subreddit':y_under})
train_df_balanced['label'] = train_df_balanced['subreddit'].map({'control': 0, 'depression': 1, 'anxiety': 2, 'bpd': 3, 'bipolar': 4})
train_df_balanced = train_df_balanced.sample(frac = 1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
train_df_balanced

,preprocessed_selftext,subreddit,label
18664,soo even feel fine day lately feel dread body ...,anxiety,2
53972,med really suppress mania like full blown mani...,bipolar,4
94783,since go back therapy little year ago therapis...,bpd,3
137325,start let say post may chaotic probably type w...,depression,1
47992,feel pretty paranoid spa think might root evil...,bipolar,4
...,...,...,...
97727,last night big mental breakdown felt angry sad...,bpd,3
99082,anybody else experience typical pm talk full b...,bpd,3
15192,totally freak nothing besides eat sure cause ...,anxiety,2
62745,know people say committedsectioned know manic ...,bipolar,4


In [ ]:
print(len(val_df[val_df['subreddit'] == 'control']))
print(len(val_df[val_df['subreddit'] == 'anxiety']))
print(len(val_df[val_df['subreddit'] == 'depression']))
print(len(val_df[val_df['subreddit'] == 'bpd']))
print(len(val_df[val_df['subreddit'] == 'bipolar']))

8142
4138
4323
3757
4452


In [ ]:
undersample = RandomUnderSampler(sampling_strategy = {'depression': 4000, 'control': 4000, 'anxiety': 4000, 'ocd': 0, 'bpd': 4000, 'bipolar': 4000}, random_state=42)
X_under, y_under = undersample.fit_resample(val_df[['preprocessed_selftext']], val_df['subreddit'])
val_df_balanced = pd.DataFrame({'preprocessed_selftext':X_under[ : , 0], 'subreddit':y_under})
val_df_balanced['label'] = val_df_balanced['subreddit'].map({'depression': 1, 'control': 0, 'anxiety': 2, 'ocd': 5, 'bpd': 3, 'bipolar': 4})
val_df_balanced = val_df_balanced.sample(frac = 1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

count_vectorizer = CountVectorizer()
count_vectorizer.fit(train_df_balanced['preprocessed_selftext'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
x_train = count_vectorizer.transform(train_df_balanced['preprocessed_selftext'])
y_train = train_df_balanced['label'].values

In [ ]:
x_val = count_vectorizer.transform(val_df_balanced['preprocessed_selftext'])
y_val = val_df_balanced['label'].values

In [ ]:
x_val.shape

(60000, 81720)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

classifier = SVC()
classifier.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred = classifier.predict(x_val)
print(classification_report(y_val, y_pred))
#print(precision_score(y_train, y_pred))
#print(recall_score(y_train, y_pred))
#print(f1_score(y_train, y_pred))

In [ ]:
print(confusion_matrix(y_val, y_pred))

In [ ]:
print(len(test_df[test_df['subreddit'] == 'control']))
print(len(test_df[test_df['subreddit'] == 'anxiety']))
print(len(test_df[test_df['subreddit'] == 'depression']))
print(len(test_df[test_df['subreddit'] == 'bpd']))
print(len(test_df[test_df['subreddit'] == 'bipolar']))

45698
23927
24333
23975
16265


In [ ]:
undersample = RandomUnderSampler(sampling_strategy = {'control': 4000, 'depression': 4000, 'anxiety': 4000, 'bpd': 4000, 'bipolar': 4000}, random_state=42)
X_under, y_under = undersample.fit_resample(test_df[['preprocessed_selftext']], test_df['subreddit'])
test_df_balanced = pd.DataFrame({'preprocessed_selftext':X_under[ : , 0], 'subreddit':y_under})
test_df_balanced['label'] = test_df_balanced['subreddit'].map({'control': 0, 'depression': 1, 'anxiety': 2, 'bpd': 3, 'bipolar': 4})
test_df_balanced = test_df_balanced.sample(frac=1)
test_df_balanced

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


,preprocessed_selftext,subreddit,label
6174,w bf year compatible matespartners great team ...,bipolar,4
6541,although day time fill beautiful thing make li...,bipolar,4
8959,opening someone something personal respond lo...,bpd,3
3526,doctor prescribed paxil mg today say look chan...,anxiety,2
17770,year bad last honest year surround family yet ...,depression,1
...,...,...,...
1508,hi already post relationship know tie anxiety ...,anxiety,2
14582,back may complex flood terribly car flood cert...,control,0
19774,edit know post right sub confessional first th...,depression,1
2272,tw medication near miss car accident hi everyo...,anxiety,2


In [ ]:
x_test = count_vectorizer.transform(test_df_balanced['preprocessed_selftext'])
y_test = test_df_balanced['label'].values

In [ ]:
y_pred = classifier.predict(x_test)
print(classification_report(y_test, y_pred))
#print(precision_score(y_train, y_pred))
#print(recall_score(y_train, y_pred))
#print(f1_score(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.92      0.86      4000
           1       0.66      0.78      0.72      4000
           2       0.83      0.78      0.80      4000
           3       0.79      0.69      0.74      4000
           4       0.82      0.71      0.76      4000

    accuracy                           0.78     20000
   macro avg       0.78      0.78      0.78     20000
weighted avg       0.78      0.78      0.78     20000



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[3695  128   70   68   39]
 [ 226 3122  200  272  180]
 [ 227  388 3108  134  143]
 [ 220  584  184 2753  259]
 [ 236  490  176  252 2846]]


In [ ]:
sum(sum(confusion_matrix(y_test, y_pred)))

20000

**Feed forward neural network**

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Individual Project/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
x_train = train_df_balanced['preprocessed_selftext'].values
y_train = train_df_balanced['label'].values

In [ ]:
len(x_train)

250000

In [ ]:
embeddings = []
for i in range(0, len(x_train)):
  embedding_sum = np.zeros(300)
  words = x_train[i].split(' ')
  for word in words:
    if word in model:
      embedding_sum = embedding_sum + model[word]
  embedding_sum = embedding_sum / len(words)

  embeddings.append(embedding_sum)

In [ ]:
len(x_train)

250000

In [ ]:
x_train_embeddings = np.array(embeddings)

In [ ]:
x_train_embeddings.shape

(250000, 300)

In [ ]:
x_train_embeddings[0][12 : 45]

array([ 0.02527214,  0.0377664 , -0.05615257,  0.04805621,  0.05744665,
        0.04408174, -0.06353401, -0.07883857,  0.06381944,  0.02764803,
        0.0613143 ,  0.02490952,  0.0204755 ,  0.00608377, -0.04036937,
        0.02257386,  0.03334674, -0.00764735, -0.04194013,  0.03487621,
       -0.02846123, -0.02564778, -0.04874555, -0.04200116,  0.03236142,
        0.00402114,  0.00728473,  0.09419161,  0.06982422, -0.09816607,
        0.19156422,  0.04734892,  0.00405704])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train_embeddings)

x_train_embeddings = scaler.transform(x_train_embeddings)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class DepressionDataset(Dataset):

   def __init__(self, x, y):
      self.x = x
      self.y = y
      
   def __len__(self):
      return len(self.x)
      
   def __getitem__(self, idx):
      return self.x[idx], self.y[idx]

In [ ]:
train = DepressionDataset(x_train_embeddings, y_train)
train_dataloader = DataLoader(train, batch_size=207)

In [ ]:
class MLP(nn.Module):
    # Define entities containing model weights in the constructor.
    def __init__(self, n_hidden, dropout_p):
        super().__init__()
        self.linear1 = nn.Linear(
            in_features=300, out_features=n_hidden, bias=True
        )
        self.linear2 = nn.Linear(
            in_features=n_hidden, out_features=n_hidden, bias=True
        )
        self.linear3 = nn.Linear(
            in_features=n_hidden, out_features=5, bias=True
        )
        self.dropout = nn.Dropout(dropout_p)

    # Then, all you need to do is implement a `forward` method to define the
    # computation that takes place on the forward pass. A corresponding
    # `backward` method, which computes gradients, is automatically defined!
    def forward(self, inputs):
        h = self.linear1(inputs)
        h = self.dropout(h)
        h = torch.relu(h)
        h = self.linear2(h)
        h = self.dropout(h)
        h = torch.relu(h)
        out = self.linear3(h)

        return out

In [ ]:
def train_model(model, train_loader, optimizer, epoch, log_interval=500):
    """
    A utility function that performs a basic training loop.

    For each batch in the training set, fetched using `train_loader`:
        - Zeroes the gradient used by `optimizer`
        - Performs forward pass through `model` on the given batch
        - Computes loss on batch
        - Performs backward pass
        - `optimizer` updates model parameters using computed gradient

    Prints the training loss on the current batch every `log_interval` batches.
    """
    model.train()

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # We need to send our batch to the device we are using. If this is not
        # it will default to using the CPU.
        inputs = inputs.to(device)
        targets = targets.to(device)

        inputs = inputs.float()
        targets = targets.float()
        
        # Zeroes the gradient used by `optimizer`; NOTE: if this is not done,
        # then gradients will be accumulated across batches!
        optimizer.zero_grad()

        # Performs forward pass through `model` on the given batch; equivalent
        # to `model.forward(inputs)`. Any information needed to compute
        # gradients is automatically thanks to autograd running under the hood.
        #print("ha")
        outputs = model(inputs)
        #print("ha")
        #print(inputs.size())
        #print(targets.size())
        #print(outputs.size())

        loss_fn = nn.CrossEntropyLoss()
        targets = targets.long()
        loss = loss_fn(outputs, targets)

        # Performs backward pass; steps backward through the computation graph,
        # computing the gradient of the loss wrt model parameters.
        loss.backward()

        # `optimizer` updates model parameters using computed gradient.
        optimizer.step()

        # Prints the training loss on the current batch every `log_interval`
        # batches.
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {:02d} -- Batch: {:03d} -- Loss: {:.4f}".format(
                    epoch,
                    batch_idx,
                    # Calling `loss.item()` returns the scalar loss as a Python
                    # number.
                    loss.item(),
                )
            )

In [ ]:
classifier = MLP(n_hidden=199, dropout_p=0.01).to(device)
optimizer = optim.SGD(classifier.parameters(), lr=0.07)
# Train-test loop
for epoch in range(1):
  train_model(classifier, train_dataloader, optimizer, epoch)

Train Epoch: 00 -- Batch: 000 -- Loss: 1.6146
Train Epoch: 00 -- Batch: 500 -- Loss: 1.0962
Train Epoch: 00 -- Batch: 1000 -- Loss: 1.0673


In [ ]:
x_test = test_df_balanced['preprocessed_selftext'].values
y_test = test_df_balanced['label'].values

embeddings = []
for i in range(0, len(x_test)):
  embedding_sum = np.zeros(300)
  words = x_test[i].split(' ')
  for word in words:
    if word in model:
      embedding_sum = embedding_sum + model[word]
  embedding_sum = embedding_sum / len(words)

  embeddings.append(embedding_sum)

x_test_embeddings = np.array(embeddings)

In [ ]:
x_test_embeddings.shape

(80000, 300)

In [ ]:
x_test_embeddings = scaler.transform(x_test_embeddings)

In [ ]:
classifier.eval()
y_pred = classifier(torch.from_numpy(x_test_embeddings).float().to(device))
y_pred = y_pred.cpu().detach().numpy()
outputs = []
for i in range(0, len(y_pred)):
  outputs.append(y_pred[i].argmax())
outputs = np.array(outputs)

In [ ]:
outputs

array([3, 2, 1, ..., 0, 2, 0])

In [ ]:
print(classification_report(y_test, outputs))

              precision    recall  f1-score   support

           0       0.71      0.84      0.77     16000
           1       0.46      0.64      0.53     16000
           2       0.62      0.55      0.58     16000
           3       0.58      0.45      0.51     16000
           4       0.62      0.47      0.54     16000

    accuracy                           0.59     80000
   macro avg       0.60      0.59      0.59     80000
weighted avg       0.60      0.59      0.59     80000



In [ ]:
print(confusion_matrix(y_train, outputs))

In [ ]:
def bayesian_train_model(n_hidden, dropout_p, batch_size, lr):
  classifier = MLP(n_hidden=int(n_hidden), dropout_p=dropout_p).to(device)
  optimizer = optim.SGD(classifier.parameters(), lr=lr)
  
  train = DepressionDataset(x_train_embeddings, y_train)
  train_dataloader = DataLoader(train, batch_size=int(batch_size))

  for epoch in range(100):
    train_model(classifier, train_dataloader, optimizer, epoch)

  classifier.eval()
  y_pred = classifier(torch.from_numpy(x_test_embeddings).float().to(device))
  y_pred = y_pred.cpu().detach().numpy()
  outputs = []
  for i in range(0, len(y_pred)):
    outputs.append(y_pred[i].argmax())
  outputs = np.array(outputs)

  return accuracy_score(y_test, outputs)

In [ ]:
bounds = {
    'n_hidden':(50, 250),
    'dropout_p': (0, 0.5),
    'batch_size':(64, 512),
    'lr':(0.001, 0.1)
}

In [ ]:
bayesian_optimizer = BayesianOptimization(
    f=bayesian_train_model,
    pbounds=bounds,
    random_state=1,
)
bayesian_optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | batch_... | dropout_p |    lr     | n_hidden  |
-------------------------------------------------------------------------
Train Epoch: 00 -- Batch: 000 -- Loss: 1.6205
Train Epoch: 00 -- Batch: 500 -- Loss: 1.5998
Train Epoch: 01 -- Batch: 000 -- Loss: 1.5949
Train Epoch: 01 -- Batch: 500 -- Loss: 1.5744
Train Epoch: 02 -- Batch: 000 -- Loss: 1.5789
Train Epoch: 02 -- Batch: 500 -- Loss: 1.5399
Train Epoch: 03 -- Batch: 000 -- Loss: 1.5538
Train Epoch: 03 -- Batch: 500 -- Loss: 1.5037
Train Epoch: 04 -- Batch: 000 -- Loss: 1.5163
Train Epoch: 04 -- Batch: 500 -- Loss: 1.4709
Train Epoch: 05 -- Batch: 000 -- Loss: 1.4804
Train Epoch: 05 -- Batch: 500 -- Loss: 1.4311
Train Epoch: 06 -- Batch: 000 -- Loss: 1.4492
Train Epoch: 06 -- Batch: 500 -- Loss: 1.4000
Train Epoch: 07 -- Batch: 000 -- Loss: 1.4252
Train Epoch: 07 -- Batch: 500 -- Loss: 1.3704
Train Epoch: 08 -- Batch: 000 -- Loss: 1.4248
Train Epoch: 08 -- Batch: 500 -- Loss: 1.3772
Train Epoch: 09 -- Batch

In [ ]:
bayesian_optimizer.max

**CNN**

In [ ]:
x_train = train_df_balanced['preprocessed_selftext'].values
y_train = train_df_balanced['label'].values

In [ ]:
x_train.shape

(170000,)

In [ ]:
vocab = {}
idx = 1
for i in range(0, len(x_train)):
  words = x_train[i].split(' ')
  for word in words:
    if word in model and word not in vocab:
      vocab[word] = idx
      idx += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  


In [ ]:
len(vocab)

118870

In [ ]:
# convert each training document into list of word indices
x_train_numeric = []
max_length = 0
for i in range(0, len(x_train)):
  word_indices = []
  words = x_train[i].split(' ')
  for word in words:
    if word in vocab:
      word_indices.append(vocab[word])
  
  x_train_numeric.append(word_indices)
  if len(word_indices) > max_length:
    max_length = len(word_indices)

In [ ]:
len(x_train_numeric[0])

119

In [ ]:
max_length = 400

In [ ]:
# add padding
for i in range(0, len(x_train_numeric)):
  num_zeros = max_length - len(x_train_numeric[i])
  if num_zeros < 0:
    x_train_numeric[i] = x_train_numeric[i][ : num_zeros]
  else:
    for j in range(0, num_zeros):
      x_train_numeric[i].append(0)

In [ ]:
x_train_numeric = np.array(x_train_numeric)

In [ ]:
x_train_numeric.shape

(170000, 400)

In [ ]:
weights_matrix = np.zeros((len(vocab) + 1, 300))
for word in vocab:
  weights_matrix[vocab[word]] = model[word]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
class CNN(nn.Module):
  def __init__(self, seq_len, num_words, embedding_size, num_filters_1, num_filters_2, num_filters_3, dropout_p):
      super().__init__()

      self.seq_len = seq_len
      self.num_words = num_words
      self.embedding_size = embedding_size

      self.dropout = nn.Dropout(dropout_p)
      self.num_filters_1 = num_filters_1
      self.num_filters_2 = num_filters_2
      self.num_filters_3 = num_filters_3

      # kernel sizes
      self.kernel_1 = 2
      self.kernel_2 = 3
      self.kernel_3 = 4
  
      self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
      self.embedding.load_state_dict({'weight': torch.from_numpy(weights_matrix)})
      self.embedding.weight.requires_grad = False

      # Convolution layers definition
      self.conv_1 = nn.Conv1d(self.embedding_size, self.num_filters_1, kernel_size=self.kernel_1) # bigrams
      self.conv_2 = nn.Conv1d(self.embedding_size, self.num_filters_2, kernel_size=self.kernel_2) # trigrams
      self.conv_3 = nn.Conv1d(self.embedding_size, self.num_filters_3, kernel_size=self.kernel_3) # 4-grams

      # Max pooling layers definition
      self.pool_1 = nn.MaxPool1d(self.seq_len - self.kernel_1 + 1)
      self.pool_2 = nn.MaxPool1d(self.seq_len - self.kernel_2 + 1)
      self.pool_3 = nn.MaxPool1d(self.seq_len - self.kernel_3 + 1)

      self.fc1 = nn.Linear(self.num_filters_1 + self.num_filters_2 + self.num_filters_3, 50)
      self.fc2 = nn.Linear(50, 5)

  def forward(self, x):
      x = x.int()
      x = self.embedding(x)
      x = x.permute(0, 2, 1)

      # Convolution layer 1 is applied
      x1 = self.conv_1(x)
      x1 = torch.relu(x1)
      x1 = self.pool_1(x1)

      # Convolution layer 2 is applied
      x2 = self.conv_2(x)
      x2 = torch.relu((x2))
      x2 = self.pool_2(x2)

      # Convolution layer 3 is applied
      x3 = self.conv_3(x)
      x3 = torch.relu(x3)
      x3 = self.pool_3(x3)

      # The output of each convolutional layer is concatenated into a unique vector
      union = torch.cat((x1, x2, x3), 1)
      union = union.reshape(union.size(0), -1)

      # The "flattened" vector is passed through a fully connected layer
      h = self.fc1(union)
      # Dropout is applied		
      h = self.dropout(h)
      h = torch.relu(h)

      out = self.fc2(h)
      # Activation function is applied
      #out = torch.softmax(out)

      return out

In [ ]:
#one_hot = torch.nn.functional.one_hot(torch.from_numpy(y_train)).long()
train = DepressionDataset(x_train_numeric, y_train)
train_dataloader = DataLoader(train, batch_size=415)

In [ ]:
classifier = CNN(seq_len=max_length, num_words=len(vocab), embedding_size=300, num_filters_1=600, num_filters_2=500, num_filters_3=300, dropout_p=0.4).to(device)
optimizer = optim.Adam(classifier.parameters(), lr=0.0001)
# Train-test loop
for epoch in range(10):
  train_model(classifier, train_dataloader, optimizer, epoch)


Train Epoch: 00 -- Batch: 000 -- Loss: 1.6789
Train Epoch: 01 -- Batch: 000 -- Loss: 0.6664
Train Epoch: 02 -- Batch: 000 -- Loss: 0.5964
Train Epoch: 03 -- Batch: 000 -- Loss: 0.5762
Train Epoch: 04 -- Batch: 000 -- Loss: 0.5724
Train Epoch: 05 -- Batch: 000 -- Loss: 0.5637
Train Epoch: 06 -- Batch: 000 -- Loss: 0.5304
Train Epoch: 07 -- Batch: 000 -- Loss: 0.4720
Train Epoch: 08 -- Batch: 000 -- Loss: 0.4728
Train Epoch: 09 -- Batch: 000 -- Loss: 0.4479


In [ ]:
x_test = val_df_balanced['preprocessed_selftext'].values
y_test = val_df_balanced['label'].values

In [ ]:
# convert each training document into list of word indices
x_test_numeric = []

for i in range(0, len(x_test)):
  word_indices = []
  words = x_test[i].split(' ')
  for word in words:
    if word in vocab:
      word_indices.append(vocab[word])
  
  x_test_numeric.append(word_indices)

In [ ]:
# add padding
for i in range(0, len(x_test_numeric)):
  num_zeros = max_length - len(x_test_numeric[i])
  if num_zeros < 0:
    x_test_numeric[i] = x_test_numeric[i][ : num_zeros]
  else:
    for j in range(0, num_zeros):
      x_test_numeric[i].append(0)


In [ ]:
x_test_numeric = np.array(x_test_numeric)

In [ ]:
x_test_numeric.shape

(20000, 400)

In [ ]:
test = DepressionDataset(x_test_numeric, y_test)
test_dataloader = DataLoader(test, batch_size=415)

In [ ]:
def get_model_outputs(model, test_loader):
    model.eval()
    outputs = []
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        # We need to send our batch to the device we are using. If this is not
        # it will default to using the CPU.
        inputs = inputs.to(device)
        
        y_pred = model(inputs)
        y_pred = y_pred.detach().cpu().numpy()
        for i in range(0, len(y_pred)):
          outputs.append(y_pred[i].argmax())


    return np.array(outputs)

In [ ]:
y_pred = get_model_outputs(classifier, train_dataloader)

In [ ]:
len(y_pred)

170000

In [ ]:
len(y_test)

20000

In [ ]:
print(classification_report(y_train, y_pred))
#print(precision_score(y_val, y_pred))
#print(recall_score(y_val, y_pred))
#print(f1_score(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97     34000
           1       0.86      0.90      0.88     34000
           2       0.92      0.91      0.92     34000
           3       0.88      0.89      0.89     34000
           4       0.93      0.86      0.90     34000

    accuracy                           0.91    170000
   macro avg       0.91      0.91      0.91    170000
weighted avg       0.91      0.91      0.91    170000



In [ ]:
print(confusion_matrix(y_test, y_pred))

ValueError: ignored

In [ ]:
def bayesian_train_model(num_filters_1, num_filters_2, num_filters_3, dropout_p, batch_size, lr):
  classifier = CNN(seq_len=max_length, num_words=len(vocab), embedding_size=300, num_filters_1=int(num_filters_1), num_filters_2=int(num_filters_2), num_filters_3=int(num_filters_3), dropout_p=dropout_p).to(device)
  optimizer = optim.Adam(classifier.parameters(), lr=lr)

  train = DepressionDataset(x_train_numeric, y_train)
  train_dataloader = DataLoader(train, batch_size=int(batch_size))

  # Train-test loop
  for epoch in range(10):
    train_model(classifier, train_dataloader, optimizer, epoch)
  
  test = DepressionDataset(x_test_numeric, y_test)
  test_dataloader = DataLoader(test, batch_size=int(batch_size))

  y_pred = get_model_outputs(classifier, test_dataloader)

  return accuracy_score(y_test, y_pred)

In [ ]:
bounds = {
    'num_filters_1':(50, 600),
    'num_filters_2':(50, 600),
    'num_filters_3':(50, 600),
    'dropout_p': (0, 0.5),
    'batch_size':(64, 512),
    'lr':(0.0001, 0.01),
}

In [ ]:
bayesian_optimizer = BayesianOptimization(
    f=bayesian_train_model,
    pbounds=bounds,
    random_state=1,
)
bayesian_optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | batch_... | dropout_p |    lr     | num_fi... | num_fi... | num_fi... |
-------------------------------------------------------------------------------------------------
Train Epoch: 00 -- Batch: 000 -- Loss: 1.7118
Train Epoch: 00 -- Batch: 500 -- Loss: 0.6032
Train Epoch: 01 -- Batch: 000 -- Loss: 0.6889
Train Epoch: 01 -- Batch: 500 -- Loss: 0.5562
Train Epoch: 02 -- Batch: 000 -- Loss: 0.6041
Train Epoch: 02 -- Batch: 500 -- Loss: 0.4901
Train Epoch: 03 -- Batch: 000 -- Loss: 0.6111
Train Epoch: 03 -- Batch: 500 -- Loss: 0.4657
Train Epoch: 04 -- Batch: 000 -- Loss: 0.5455
Train Epoch: 04 -- Batch: 500 -- Loss: 0.4648
Train Epoch: 05 -- Batch: 000 -- Loss: 0.5055
Train Epoch: 05 -- Batch: 500 -- Loss: 0.4336
Train Epoch: 06 -- Batch: 000 -- Loss: 0.5128
Train Epoch: 06 -- Batch: 500 -- Loss: 0.4171
Train Epoch: 07 -- Batch: 000 -- Loss: 0.4609
Train Epoch: 07 -- Batch: 500 -- Loss: 0.4133
Train Epoch: 08 -- Batch: 000 -- Loss: 0.4315
Train Epoch: 08 -- Bat

In [ ]:
bayesian_optimizer.max

{'params': {'batch_size': 512.0,
  'dropout_p': 0.45390976039702363,
  'lr': 0.0001,
  'num_filters_1': 583.5689343238934,
  'num_filters_2': 510.1024441228828,
  'num_filters_3': 298.41850117229467},
 'target': 0.8141}

**Train my own word2vec embeddings**

In [ ]:
posts = train_df_balanced['preprocessed_selftext'].values
word2vec_corpus = []
for i in range(0, len(posts)):
  word2vec_corpus.append(posts[i].split(' '))

In [ ]:
len(word2vec_corpus)

170000

In [ ]:
model = gensim.models.Word2Vec(sentences=word2vec_corpus, size=300, window=5, min_count=1, workers=4, iter=20)

**LSTM**

In [ ]:
class LSTM(nn.Module):
  def __init__(self, num_words, embedding_size, hidden_dim, num_lstm_layers, dropout_p):
    super().__init__()

    self.num_words = num_words
    self.embedding_size = embedding_size
    self.hidden_dim = hidden_dim
    self.num_lstm_layers = num_lstm_layers

    self.dropout = nn.Dropout(dropout_p)

    self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
    self.embedding.load_state_dict({'weight': torch.from_numpy(weights_matrix)})
    self.embedding.weight.requires_grad = False

    self.lstm = nn.LSTM(input_size=self.embedding_size, hidden_size=self.hidden_dim, num_layers=self.num_lstm_layers, batch_first=True)
    self.fc1 = nn.Linear(in_features=self.hidden_dim, out_features=5)
    #self.fc2 = nn.Linear(in_features=self.hidden_dim // 2, out_features=5)

  def forward(self, x):
    h = torch.zeros((self.num_lstm_layers, x.size(0), self.hidden_dim)).to(device)
    c = torch.zeros((self.num_lstm_layers, x.size(0), self.hidden_dim)).to(device)

    #torch.nn.init.xavier_normal_(h)
    #torch.nn.init.xavier_normal_(c)
    
    x = x.int()
    out = self.embedding(x)
    print(out.size())
    out, (hidden, cell) = self.lstm(out, (h,c))
    out = self.dropout(out)

    #out = torch.relu_(self.fc1(out[:,-1,:]))
    #out = self.dropout(out)
    #out = self.fc2(out)

    out = self.fc1(out[:,-1,:])

    return out

In [ ]:
classifier = LSTM(num_words=len(vocab), embedding_size=300, hidden_dim=75, num_lstm_layers=2, dropout_p=0).to(device)
optimizer = optim.Adam(classifier.parameters(), lr=0.003)
# Train-test loop
for epoch in range(10):
  train_model(classifier, train_dataloader, optimizer, epoch)


torch.Size([300, 400, 300])
Train Epoch: 00 -- Batch: 000 -- Loss: 1.6108
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
torch.Size([300, 400, 300])
to

KeyboardInterrupt: ignored

In [ ]:
y_pred = get_model_outputs(classifier, test_dataloader)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.88      0.90      4000
           1       0.67      0.80      0.73      4000
           2       0.81      0.81      0.81      4000
           3       0.77      0.74      0.76      4000
           4       0.83      0.76      0.79      4000

    accuracy                           0.80     20000
   macro avg       0.80      0.80      0.80     20000
weighted avg       0.80      0.80      0.80     20000



In [ ]:
def bayesian_train_model(hidden_dim, dropout_p, batch_size, lr):
  classifier = LSTM(num_words=len(vocab), embedding_size=300, hidden_dim=int(hidden_dim), num_lstm_layers=2, dropout_p=dropout_p).to(device)
  optimizer = optim.Adam(classifier.parameters(), lr=lr)

  train = DepressionDataset(x_train_numeric, y_train)
  train_dataloader = DataLoader(train, batch_size=int(batch_size))

  # Train-test loop
  for epoch in range(10):
    train_model(classifier, train_dataloader, optimizer, epoch)
  
  test = DepressionDataset(x_test_numeric, y_test)
  test_dataloader = DataLoader(test, batch_size=int(batch_size))

  y_pred = get_model_outputs(classifier, test_dataloader)

  return accuracy_score(y_test, y_pred)

In [ ]:
bounds = {
    'hidden_dim':(20, 150),
    'dropout_p': (0, 0.3),
    'batch_size':(64, 512),
    'lr':(0.0001, 0.01),
}

In [ ]:
bayesian_optimizer = BayesianOptimization(
    f=bayesian_train_model,
    pbounds=bounds,
    random_state=1,
)
bayesian_optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | batch_... | dropout_p | hidden... |    lr     |
-------------------------------------------------------------------------
Train Epoch: 00 -- Batch: 000 -- Loss: 1.6228
Train Epoch: 00 -- Batch: 500 -- Loss: 1.6117
Train Epoch: 01 -- Batch: 000 -- Loss: 1.6117
Train Epoch: 01 -- Batch: 500 -- Loss: 1.6169
Train Epoch: 02 -- Batch: 000 -- Loss: 1.6105
Train Epoch: 02 -- Batch: 500 -- Loss: 1.6167
Train Epoch: 03 -- Batch: 000 -- Loss: 1.6085
Train Epoch: 03 -- Batch: 500 -- Loss: 1.6135
Train Epoch: 04 -- Batch: 000 -- Loss: 1.6076
Train Epoch: 04 -- Batch: 500 -- Loss: 1.6149
Train Epoch: 05 -- Batch: 000 -- Loss: 1.6153
Train Epoch: 05 -- Batch: 500 -- Loss: 1.6120
Train Epoch: 06 -- Batch: 000 -- Loss: 1.6088
Train Epoch: 06 -- Batch: 500 -- Loss: 1.5294
Train Epoch: 07 -- Batch: 000 -- Loss: 1.5426
Train Epoch: 07 -- Batch: 500 -- Loss: 1.6113
Train Epoch: 08 -- Batch: 000 -- Loss: 1.6100
Train Epoch: 08 -- Batch: 500 -- Loss: 1.6118
Train Epoch: 09 -- Batch

In [ ]:
bayesian_optimizer.max

{'params': {'batch_size': 173.83987621924035,
  'dropout_p': 0.0,
  'hidden_dim': 77.5165200461393,
  'lr': 0.0008469775389387133},
 'target': 0.8095}

**CNN-LSTM**

In [ ]:
class CNN_LSTM(nn.Module):
  def __init__(self, num_words, embedding_size, num_filters_1, num_filters_2, num_filters_3, hidden_dim, num_lstm_layers, dropout_p):
    super().__init__()

    self.num_words = num_words
    self.embedding_size = embedding_size
    self.hidden_dim = hidden_dim
    self.num_lstm_layers = num_lstm_layers

    self.num_filters_1 = num_filters_1
    self.num_filters_2 = num_filters_2
    self.num_filters_3 = num_filters_3

    # kernel sizes
    self.kernel_1 = 2
    self.kernel_2 = 3
    self.kernel_3 = 4

    self.dropout = nn.Dropout(dropout_p)

    self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
    self.embedding.load_state_dict({'weight': torch.from_numpy(weights_matrix)})
    self.embedding.weight.requires_grad = False

    # Convolution layers definition
    #self.conv_1 = nn.Conv1d(self.embedding_size, self.num_filters_1, kernel_size=self.kernel_1) # bigrams
    self.conv_2 = nn.Conv1d(self.embedding_size, self.num_filters_2, kernel_size=self.kernel_2) # trigrams
    #self.conv_3 = nn.Conv1d(self.embedding_size, self.num_filters_3, kernel_size=self.kernel_3) # 4-grams

    self.lstm = nn.LSTM(input_size=num_filters_2, hidden_size=self.hidden_dim, num_layers=self.num_lstm_layers, batch_first=True)
    self.fc1 = nn.Linear(in_features=self.hidden_dim, out_features=5)
    #self.fc2 = nn.Linear(in_features=self.hidden_dim // 2, out_features=5)

  def forward(self, x):
    h = torch.zeros((self.num_lstm_layers, x.size(0), self.hidden_dim)).to(device)
    c = torch.zeros((self.num_lstm_layers, x.size(0), self.hidden_dim)).to(device)

    #torch.nn.init.xavier_normal_(h)
    #torch.nn.init.xavier_normal_(c)

    x = x.int()
    x = self.embedding(x)
    x = x.permute(0, 2, 1)

    # Convolution layer 1 is applied
    #x1 = self.conv_1(x)
    #x1 = torch.relu(x1)
    #print(x1.size())

    # Convolution layer 2 is applied
    x2 = self.conv_2(x)
    x2 = torch.relu((x2))
    #print(x2.size())

    # Convolution layer 3 is applied
    #x3 = self.conv_3(x)
    #x3 = torch.relu(x3)
    #print(x3.size()[2])

    #x1 = x1[ : , : , : x3.size()[2]]
    #x2 = x2[ : , : , : x3.size()[2]]

    #print(x1.size())
    #print(x2.size())
    #print(x3.size())

    # The output of each convolutional layer is concatenated into a unique vector
    #union = torch.cat((x1, x2, x3), 1)
    #union = union.reshape(union.size(0), -1)

    out = x2.permute(0, 2, 1)

    
    #x = x.int()
    #out = self.embedding(x)
    out, (hidden, cell) = self.lstm(out, (h,c))
    out = self.dropout(out)

    #out = torch.relu_(self.fc1(out[:,-1,:]))
    #out = self.dropout(out)
    #out = self.fc2(out)
    
    out = self.fc1(out[:,-1,:])

    return out

In [ ]:
classifier = CNN_LSTM(num_words=len(vocab), embedding_size=300, num_filters_1=50, num_filters_2=215, num_filters_3=50, hidden_dim=160, num_lstm_layers=2, dropout_p=0.02).to(device)
optimizer = optim.Adam(classifier.parameters(), lr=0.005)
# Train-test loop
for epoch in range(10):
  train_model(classifier, train_dataloader, optimizer, epoch)
"""
|  39       |  0.8096   |  416.5    |  0.02754  |  159.9    |  0.003682 |  214.3    |
|   iter    |  target   | batch_... | dropout_p | hidden... |    lr     | num_fi... |
"""

Train Epoch: 00 -- Batch: 000 -- Loss: 1.6106
Train Epoch: 01 -- Batch: 000 -- Loss: 1.6097
Train Epoch: 02 -- Batch: 000 -- Loss: 1.6099
Train Epoch: 03 -- Batch: 000 -- Loss: 1.6097
Train Epoch: 04 -- Batch: 000 -- Loss: 0.8902
Train Epoch: 05 -- Batch: 000 -- Loss: 0.6725
Train Epoch: 06 -- Batch: 000 -- Loss: 0.6210
Train Epoch: 07 -- Batch: 000 -- Loss: 0.6149
Train Epoch: 08 -- Batch: 000 -- Loss: 0.5791
Train Epoch: 09 -- Batch: 000 -- Loss: 0.5745


'\n|  39       |  0.8096   |  416.5    |  0.02754  |  159.9    |  0.003682 |  214.3    |\n|   iter    |  target   | batch_... | dropout_p | hidden... |    lr     | num_fi... |\n'

In [ ]:
y_pred = get_model_outputs(classifier, test_dataloader)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.92      0.90      4000
           1       0.70      0.76      0.73      4000
           2       0.78      0.84      0.81      4000
           3       0.81      0.67      0.73      4000
           4       0.79      0.77      0.78      4000

    accuracy                           0.79     20000
   macro avg       0.79      0.79      0.79     20000
weighted avg       0.79      0.79      0.79     20000



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[3716  100   55   58   71]
 [ 102 2993  246  390  269]
 [ 130  293 3263  153  161]
 [  94  388  174 3014  330]
 [ 100  334  153  283 3130]]


In [ ]:
def bayesian_train_model(num_filters_2, hidden_dim, dropout_p, batch_size, lr, kernel_2):
  classifier = CNN_LSTM(num_words=len(vocab), embedding_size=300, num_filters_1=10, num_filters_2=int(num_filters_2), num_filters_3=10, hidden_dim=int(hidden_dim), dropout_p=dropout_p, num_lstm_layers=2, kernel_2=int(kernel_2)).to(device)
  optimizer = optim.Adam(classifier.parameters(), lr=lr)

  train = DepressionDataset(x_train_numeric, y_train)
  train_dataloader = DataLoader(train, batch_size=int(batch_size))

  # Train-test loop
  for epoch in range(10):
    train_model(classifier, train_dataloader, optimizer, epoch)
  
  test = DepressionDataset(x_test_numeric, y_test)
  test_dataloader = DataLoader(test, batch_size=int(batch_size))

  y_pred = get_model_outputs(classifier, test_dataloader)

  return accuracy_score(y_test, y_pred)

In [ ]:
bounds = {
    'hidden_dim':(50, 200),
    'dropout_p': (0, 0.3),
    'batch_size':(64, 512),
    'lr':(0.0001, 0.01),
    'num_filters_2': (50, 300),
    'kernel_2': (2, 5)
}

In [ ]:
bayesian_optimizer = BayesianOptimization(
    f=bayesian_train_model,
    pbounds=bounds,
    random_state=1,
)
bayesian_optimizer.maximize(init_points=10, n_iter=50)
|  50       |  0.8079   |  310.3    |  0.2355   |  123.1    |  3.047    |  0.002983 |  104.0    |
|   iter    |  target   | batch_... | dropout_p | hidden... | kernel_2  |    lr     | num_fi... |

|   iter    |  target   | batch_... | dropout_p | hidden... | kernel_2  |    lr     | num_fi... |
-------------------------------------------------------------------------------------------------
Train Epoch: 00 -- Batch: 000 -- Loss: 1.6108
Train Epoch: 00 -- Batch: 500 -- Loss: 1.6112
Train Epoch: 01 -- Batch: 000 -- Loss: 1.6094
Train Epoch: 01 -- Batch: 500 -- Loss: 1.6106
Train Epoch: 02 -- Batch: 000 -- Loss: 1.5075
Train Epoch: 02 -- Batch: 500 -- Loss: 1.4977
Train Epoch: 03 -- Batch: 000 -- Loss: 1.6075
Train Epoch: 03 -- Batch: 500 -- Loss: 1.5954
Train Epoch: 04 -- Batch: 000 -- Loss: 1.3998
Train Epoch: 04 -- Batch: 500 -- Loss: 1.4372
Train Epoch: 05 -- Batch: 000 -- Loss: 1.3330
Train Epoch: 05 -- Batch: 500 -- Loss: 1.2378
Train Epoch: 06 -- Batch: 000 -- Loss: 1.1091
Train Epoch: 06 -- Batch: 500 -- Loss: 1.0491
Train Epoch: 07 -- Batch: 000 -- Loss: 0.7913
Train Epoch: 07 -- Batch: 500 -- Loss: 0.7850
Train Epoch: 08 -- Batch: 000 -- Loss: 0.6558
Train Epoch: 08 -- Bat

**n-gram language model**


In [ ]:
from nltk import bigrams, trigrams
from collections import Counter, defaultdict


In [ ]:
def build_bigram_language_model(subreddit):
  model = defaultdict(lambda: defaultdict(lambda: 0))
  posts = train_df_balanced[train_df_balanced['subreddit'] == subreddit]['preprocessed_selftext'].values
  tokenised_posts = []
  for i in range(0, len(posts)):
    words = posts[i].split(' ')
    tokenised_posts.append(words)

  vocabulary = {}
  for post in tokenised_posts:
    for word in post:
      if word not in vocabulary:
        vocabulary[word] = 1

  for post in tokenised_posts:
    for w1, w2 in bigrams(post, pad_right=True, pad_left=True):
      model[w1][w2] += 1

  for w1 in model:
      total_count = float(sum(model[w1].values()))
      for w2 in model[w1]:
        model[w1][w2] = (model[w1][w2] + 1) / (total_count + len(vocabulary))

  return model, vocabulary


In [ ]:
def build_trigram_language_model(subreddit):
  model = defaultdict(lambda: defaultdict(lambda: 0))
  posts = train_df_balanced[train_df_balanced['subreddit'] == subreddit]['preprocessed_selftext'].values
  tokenised_posts = []
  for i in range(0, len(posts)):
    words = posts[i].split(' ')
    tokenised_posts.append(words)

  vocabulary = {}
  for post in tokenised_posts:
    for word in post:
      if word not in vocabulary:
        vocabulary[word] = 1

  for post in tokenised_posts:
    for w1, w2, w3 in trigrams(post, pad_right=True, pad_left=True):
      model[(w1, w2)][w3] += 1

  for w1_w2 in model:
      total_count = float(sum(model[w1_w2].values()))
      for w3 in model[w1_w2]:
        model[w1_w2][w3] = (model[w1_w2][w3] + 1) / (total_count + len(vocabulary))

  return model, vocabulary


In [ ]:
control_model_b, control_vocab_b = build_bigram_language_model('control')
depression_model_b, depression_vocab_b = build_bigram_language_model('depression')
anxiety_model_b, anxiety_vocab_b = build_bigram_language_model('anxiety')
bpd_model_b, bpd_vocab_b = build_bigram_language_model('bpd')
bipolar_model_b, bipolar_vocab_b = build_bigram_language_model('bipolar')

In [ ]:
control_model_t, control_vocab_t = build_trigram_language_model('control')
depression_model_t, depression_vocab_t = build_trigram_language_model('depression')
anxiety_model_t, anxiety_vocab_t = build_trigram_language_model('anxiety')
bpd_model_t, bpd_vocab_t = build_trigram_language_model('bpd')
bipolar_model_t, bipolar_vocab_t = build_trigram_language_model('bipolar')

In [ ]:
import math

def get_log_prob_b(post, model, vocabulary):
  log_prob = 0
  words = post.split(' ')
  if len(words) < 2:
    return 0
  for i in range(1, len(words)):
    if words[i - 1] in model and words[i] in model[words[i - 1]]:
      log_prob += math.log(model[words[i - 1]][words[i]])
    else:
      log_prob += math.log(1 / len(vocabulary))

  return log_prob

In [ ]:
def get_log_prob_t(post, model, vocabulary):
  log_prob = 0
  words = post.split(' ')
  if len(words) < 3:
    return 0
  for i in range(2, len(words)):
    if (words[i - 2], words[i - 1]) in model and words[i] in model[(words[i - 2], words[i - 1])]:
      log_prob += math.log(model[(words[i - 2], words[i - 1])][words[i]])
    else:
      log_prob += math.log(1 / len(vocabulary))

  return log_prob